In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
import inspect
import getpass
import json

import pandas as pd
import matplotlib.pyplot as plt


project_path = os.path.abspath( os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe()))) + "/../" )
sys.path.insert(0, project_path)
print(project_path)

from database_storage.helper import *

from database_storage.database.database import *
from database_storage.main import *

from database_storage.objects.human import *
from database_storage.objects.issue import *
from database_storage.objects.kpi import *
from database_storage.objects.machine import *
from database_storage.objects.tag import *
from database_storage.objects.maintenanceworkorder import *


/Users/saschamoccozet/Documents/Git/Nestor


# Get CSVfile Path

### Greystone

In [ ]:
user = "neo4j"
password = 'GREYSTONE!!'

csv_file = os.path.join(project_path ,'data', 'gs_data', 'readable.csv')
csvSchemas_file = os.path.join(project_path , 'data',  'gs_data', 'gs_header.yaml')


bincsv_file = os.path.join(project_path ,'data', 'gs_data', 'bin_1g.csv')
binNcsv_file = os.path.join(project_path ,'data', 'gs_data', 'bin_Ng.csv')


### Mine

In [2]:
user = "neo4j"
password = 'MINE!!'

csv_file = os.path.join(project_path ,'data', 'mine_data', 'readable.csv')
csvSchemas_file = os.path.join(project_path , 'data',  'mine_data', 'mine_header.yaml')


bincsv_file = os.path.join(project_path ,'data', 'mine_data', 'bin_1g.csv')
binNcsv_file = os.path.join(project_path ,'data', 'mine_data', 'bin_Ng.csv')

vocabcsv_file = os.path.join(project_path ,'data', 'mine_data', 'vocab_1g.csv')
vocabNcsv_file = os.path.join(project_path ,'data', 'mine_data', 'vocab_2g.csv')


In [5]:
tree_path = os.path.join(project_path ,'data', 'mine_data', 'tree_test.json')

### HVAC

In [ ]:
user = "neo4j"
password = 'HVAC!!'

csv_file = os.path.join(project_path ,'data', 'hvac_data', 'readable.csv')
csvSchemas_file = os.path.join(project_path , 'data',  'hvac_data', 'hvac_header.yaml')


bincsv_file = os.path.join(project_path ,'data', 'hvac_data', 'bin_1g.csv')
binNcsv_file = os.path.join(project_path ,'data', 'hvac_data', 'bin_Ng.csv')


### Others

In [ ]:
user = getpass.getuser()
password = getpass.getpass("Password:")

csv_file = os.path.join(project_path ,'data', 'hvac_data', 'hvac_raw.csv')
csvSchemas_file = os.path.join(project_path , 'data',  'hvac_data', 'hvac_header.yaml')

## CSV to Dataframe

In [3]:
dataframe = pd.read_csv(csv_file, index_col=0, header=0)
dataframe.fillna("", inplace=True)


dataframe_bincsv = pd.read_csv(bincsv_file, index_col=0, header=[0,1])
dataframe_binNcsv = pd.read_csv(binNcsv_file, index_col=0, header=[0,1])

dataframe_vocab1g = pd.read_csv(vocabcsv_file, index_col=0)
dataframe_vocabNg = pd.read_csv(vocabNcsv_file, index_col=0)

csvSchemas_dict = openYAMLFile(csvSchemas_file)

yaml file open


In [6]:
with open(tree_path) as file:
    tree_dict = json.load(file)

# ConnectGraphDatabase

In [7]:
databaseSchema_file = os.path.join(project_path ,'database_storage','database', 'DatabaseSchema.yaml')
databaseSchema_dict = openYAMLFile(databaseSchema_file)
database = DatabaseNeo4J("bolt://127.0.0.1:7687", user, password, databaseSchema_dict)

yaml file open


## Clean the database

In [15]:
database.deleteData()

database.dropConstraints()
database.dropIndexes()

database.createIndexes()
database.createConstraints()

1

## Create Cypher Queries

### Create historical_data

In [16]:
database.runQueries(cypherCreate_historicalMaintenanceWorkOrder (database, dataframe, csvSchemas_dict))

100%|██████████| 5485/5485 [00:18<00:00, 290.27it/s]


1

### Create tag_data

In [17]:
database.runQueries(cypherCreate_tag(database, dataframe_bincsv, dataframe_vocab1g, dataframe_vocabNg))

100%|██████████| 2058/2058 [00:11<00:00, 173.63it/s]


1

In [18]:
database.runQueries(cypherCreate_tag(database, dataframe_binNcsv, dataframe_vocab1g, dataframe_vocabNg))

100%|██████████| 1401/1401 [00:06<00:00, 210.44it/s]


1

### Update 1gram-->Ngram

In [19]:
database.runQueries(cypherLink_Ngram1gram(database))

100%|██████████| 6/6 [00:00<00:00, 28.41it/s]


1

### Update issue-->item

In [20]:
database.runQueries(cypherLink_itemIssue(database))

100%|██████████| 2/2 [00:00<00:00,  7.10it/s]


1

### Update item-->item

In [21]:
database.runQueries(cypherCreate_itemsTree(database,tree_dict ))

100%|██████████| 24/24 [00:00<00:00, 223.22it/s]


1

------------------